# 测试、调试和异常

## 测试stdout

In [1]:
def urlprint(protocol, host, domain):
    url = '{}://{}.{}'.format(protocol, host, domain)
    print(url)

In [2]:
from io import StringIO
import unittest
from unittest import TestCase
from unittest.mock import patch

class TestURLPrint(TestCase):
    def test_url_gets_to_stdout(self):
        protocol = 'http'
        host = 'www'
        domain = 'example.com'
        expected_url = '{}://{}.{}\n'.format(protocol, host, domain)
        with patch('sys.stdout', new=StringIO()) as fake_out:
            urlprint(protocol, host, domain)
            self.assertEqual(fake_out.getvalue(), expected_url)

loader = unittest.TestLoader()
suite = loader.loadTestsFromTestCase(TestURLPrint)
unittest.TextTestRunner(sys.stdout,verbosity=2).run(suite)

test_url_gets_to_stdout (__main__.TestURLPrint) ...ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

## 在单元测试中给对象打补丁

In [3]:

from unittest.mock import patch

def f(x):
    print(x)

@patch('__main__.f')
def test1(x, mock_func):
    f(x)
    mock_func.assert_called_with(x)

test1(1)

In [4]:
def test2(x):
    with patch('__main__.f') as mock_func:
        f(x)
        mock_func.assert_called_with(x)

test2(1)

## 在单元测试中测试异常情况

In [5]:
import unittest

def parse_int(s):
    return int(s)

class TestConversion(unittest.TestCase):
    def test_bad_int(self):
        self.assertRaises(ValueError, parse_int, 'N/A')

loader = unittest.TestLoader()
suite = loader.loadTestsFromTestCase(TestConversion)
unittest.TextTestRunner(sys.stdout,verbosity=2).run(suite)

test_bad_int (__main__.TestConversion) ...ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [6]:
import errno

class TestIO(unittest.TestCase):
    def test_file_not_found(self):
        try:
            f = open('/file')
        except IOError as e:
            self.assertEqual(e.errno, errno.ENOENT)
        else:
            self.fail('IOError not raised')

loader = unittest.TestLoader()
suite = loader.loadTestsFromTestCase(TestIO)
unittest.TextTestRunner(sys.stdout,verbosity=2).run(suite)

test_file_not_found (__main__.TestIO) ...ok

----------------------------------------------------------------------
Ran 1 test in 0.006s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

## 将测试输出用日志文件记录

In [7]:
import unittest

def main(out=sys.stderr, verbosity=2):
    loader = unittest.TestLoader()
    suite = loader.loadTestsFromModule(sys.modules[__name__])
    unittest.TextTestRunner(out, verbosity=verbosity).run(suite)


with open('data/testing.out', 'w') as f:
    main(f)

## 忽略或期望测试失败

In [8]:
import unittest
import os
import platform

class Tests(unittest.TestCase):
    def test_0(self):
        self.assertTrue(True)

    @unittest.skip('skipped test')
    def test_1(self):
        self.fail('should have failed!')
    
    @unittest.skipIf(os.name=='posix', 'Not supported on Unix')
    def test_2(self):
        import winreg

    @unittest.skipUnless(platform.system() == 'Darwin', 'Mac specific test')
    def test_3(self):
        self.assertTrue(True)

    @unittest.expectedFailure
    def test_4(self):
        self.assertEqual(2+3, 5)

loader = unittest.TestLoader()
suite = loader.loadTestsFromTestCase(Tests)
unittest.TextTestRunner(sys.stdout,verbosity=2).run(suite)

test_0 (__main__.Tests) ...ok
test_1 (__main__.Tests) ...skipped 'skipped test'
test_2 (__main__.Tests) ...ok
test_3 (__main__.Tests) ...skipped 'Mac specific test'
test_4 (__main__.Tests) ...unexpected success

----------------------------------------------------------------------
Ran 5 tests in 0.011s

FAILED (skipped=2, unexpected successes=1)


<unittest.runner.TextTestResult run=5 errors=0 failures=0>

## 处理多个异常

In [9]:
import errno

try:
    f = open('data/xxxx')
except (FileNotFoundError, PermissionError) as e:
    if e.errno == errno.ENOENT:
        print('File not found')
    elif e.errno == errno.EACCES:
        print('Permission denied')
    else:
        print('Unexpected error: %d', e.errno)

File not found


## 捕获所有异常

In [10]:
try:
    i = 1/0
except Exception as e:
    print('Reason: ', e)

Reason:  division by zero


## 创建自定义异常

In [11]:
# 自定义异常直接继承Exception即可

class NetworkError(Exception):
    pass

class HostnameError(NetworkError):
    pass

class TimeoutError(NetworkError):
    pass

class ProtocolError(NetworkError):
    pass

In [12]:
try:
    raise ProtocolError()
except NetworkError as e:
    print('error')

error


## 捕获异常后抛出另外的异常

In [13]:
def example():
    try:
        int('N/A')
    except ValueError as e:
        raise RuntimeError('A parsing error occurred') from e


try:
    example()
except RuntimeError as e:
    print("It didn't work:", e)
    if e.__cause__:
        print('Cause:', e.__cause__)

It didn't work: A parsing error occurred
Cause: invalid literal for int() with base 10: 'N/A'


## 重新抛出被捕获的异常

In [15]:
def example():
    try:
        int('N/A')
    except ValueError:
        print("Didn't work")
        raise

try:
    example()
except ValueError as e:
    print('Cause:', e)

Didn't work
Cause: invalid literal for int() with base 10: 'N/A'
